In [1]:
using BandedMatrices, LinearAlgebra, SparseArrays, IntervalArithmetic, Plots, LaTeXStrings, Serialization

In [2]:
Base.:*(A::SparseMatrixCSC{Interval{Float64}, Int64},x::Vector{Interval{Float64}})=Vector((A*sparse(x[:,:]))[:])

In [3]:
function D(N)
    # implements ∂ₓ in Fourier
    v = interval.([ ((n+1 ) % 2) * (n÷2) for n=1:2*N])
    return dropzeros(sparse(BandedMatrix(-1 => v, 1 =>-v)))
end

function D2(N)
    # implements ∂ₓₓ in Fourier
    v = interval.([-(n÷2)^2 for n=1:2*N+1])
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 4)))[:,1:2*N+1]))
end

function id(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => v)))
end

function id2(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 4)))[:,1:2*N+1]))
end

function idθ(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 2)))[:,1:2*N+1]))
end

function C(N)
    # implements the u ↦ cosx u in Fourier
    v = vcat([interval(0.0)], ones(Interval{Float64}, 2*N+2)/interval(2))
    A = dropzeros(sparse(BandedMatrix( -2 => v, 2 => v[1:end-2])[:,1:2*N+1]))
    A[1,3] = interval(0.5)
    A[3,1] = interval(1.0)
    return A
end

function S(N)
    # implements the u ↦ sinx u in Fourier
    v = vcat([interval(0.0)],interval.([ ((n+1 ) % 2) for n=1:2*N+2]))/interval(2)
    A = dropzeros(sparse(BandedMatrix( -1 => v, -3 =>-v[2:end], 1 => v, 3=>-v[2:end])[:,1:2*N+1]))
    A[1,2] = interval(0.5)
    A[2,1] = interval(1.0)
    return A
end

function C2(N)
    # implements the u ↦ cos2x u in Fourier
    v = vcat([interval(0.0)], ones(Interval{Float64}, 2*N)/interval(2))
    A = dropzeros(sparse(BandedMatrix( -4 => v, 4 => v[2:end-1])[:,1:2*N+1]))
    # A[1,3] = interval(0.5)
    A[5,1] = interval(1.0)
    A[3,3] = interval(0.5)
    A[2,2] = -interval(0.5)
    return A
end


function S2(N)
    # implements the u ↦ sin2x u in Fourier
    v = vcat([interval(0.0)],interval.([ ((n+1 ) % 2) for n=1:2*N]))/interval(2)
    A = dropzeros(sparse(BandedMatrix( -3 => v, -5 =>-v[2:end], 3 => v, 5=>-v[2:end])[:,1:2*N+1]))
    A[1,4] = interval(0.5)
    A[4,1] = interval(1.0)
    A[2,3] = interval(0.5)
    A[3,2] = interval(0.5)
    return A
end

S2 (generic function with 1 method)

In [4]:
N = 50

50

In [5]:
Dx = (kron(S2(N),id2(N))/interval(4)-kron(S(N),C(N)))*kron(D(N),id(N));

In [6]:
Dy = (kron(id2(N),S2(N))/interval(4)+kron(C(N),S(N)))*kron(id(N),D(N));

In [7]:
Δ = kron(D2(N),id2(N))+kron(id2(N),D2(N));

In [8]:
Q = Vector((kron(C2(N), id2(N))+kron(id2(N), C2(N)))[:,1])/interval(4);

In [9]:
𝔏 = (Dx + Dy + Δ);

In [10]:
ū = interval.(mid.(𝔏[2:end,:])\mid.(Q[2:end]));

In [11]:
ϵ = 𝔏*ū - Q;
λ̄_Σ = -ϵ[1]
ϵ[1] = interval(0)
δ = sum(abs.(ϵ))
λ_Σ = λ̄_Σ +interval(-1,1)*δ

[-0.0308583, -0.0308582]_com

In [ ]:
serialize("lambda_Sigma", λ_Σ)

In [ ]:
M = 200
x = y = collect(0:M)/M*(2π)
ū_vals = zeros((M+1,M+1))
count = 1
for i=0:2*N
    for j=0:2*N
        ū_vals += mid(ū[count])*((i%2==1)*sin.((i÷2+1)*x)+(i%2==0)*cos.((i÷2)*x)).*((j%2==1)*sin.((j÷2+1)*y)+(j%2==0)*cos.((j÷2)*y))'
        count +=1
    end
end

In [ ]:
plot(x,y,ū_vals,st=:surface, xlabel = L"$x$", ylabel = L"$y$", zlabel = L"$\bar{u}$", colorbar=false, dpi = 500)

In [ ]:
png("volume_ubar")